# Fantasy Football Cheatsheet Creator
1. Take input from user based on league settings
    - number of teams in the league
    - roster details
    - ppr rules
    - add details to dictionary for later use
2. Scraping ADP
    - use BeautifulSoup scrape player ADP for league's PPR format from [FantasyPros](https://www.fantasypros.com/)
    - create data frames for each positon, ordered by ADP
3. Find replacement players based on ADP
    - use the user's league details to find the ADP of the replacement player, ready to calculate value
4. Scraping player projections
    - use BeautifulSoup scrape player projections from [FantasyPros](https://www.fantasypros.com/)
    - create a data frame for all positions, ordered by prokected fantasy points
5. Calculate value over replacement and output cheatsheet
    - use replacement player dictionary and projection data frames to calculate each player's value over the replacement
    - create data frame ordered by value over replacement
    - save cheatsheet

## Section 0: Setup

In [1]:
# imports
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BS
import requests

## Section 1: Taking Input From User

In [2]:
leagueName = input('What is the name of your league? ')
numTeams = float(input('How many teams in your league? '))
numQB = float(input('How many quarterbacks on the roster? '))
numRB = float(input('How many running backs on the roster? '))
numWR = float(input('How many wide receivers on the roster? '))
numTE = float(input('How many tight ends on the roster? '))
numSFLX = float(input('How many QB/RB/WR/TE flex on the roster? '))
numRWTFLX = float(input('How many RB/WR/TE flex on the roster? '))
numRWFLX = float(input('How many RB/WR flex on the roster? '))
numDST = float(input('How many DST on the roster? '))
numBENCH = float(input('How many bench spots? '))

What is the name of your league? Jameis WINston
How many teams in your league? 12
How many quarterbacks on the roster? 1
How many running backs on the roster? 2
How many wide receivers on the roster? 2
How many tight ends on the roster? 1
How many QB/RB/WR/TE flex on the roster? 1
How many RB/WR/TE flex on the roster? 1
How many RB/WR flex on the roster? 0
How many DST on the roster? 1
How many bench spots? 6


In [3]:
roster = {
    'QB': numQB,
    'RB': numRB,
    'WR': numWR,
    'TE': numTE,
    'QB/RB/WR/TE': numSFLX,
    'RB/WR/TE': numRWTFLX,
    'RB/WR': numRWFLX,
    'DST': numDST,
    'Bench': numBENCH
}

rostersize = sum(roster.values())
playersdrafted = numTeams * rostersize

In [4]:
ppr = input('How many points per reception? (0, 0.5 or 1) ')
if ppr not in ['0', '0.5', '1', '1.0']:
    print('Invalid PPR entered. Please try again.')
else:
    ppr = float(ppr)

How many points per reception? (0, 0.5 or 1) 1


## Section 2: Scraping FantasyPros for ADP
This will be used to identify the players whose ADP puts them at the level of a 'replacement' based on the user's roster

In [5]:
if ppr == 0:
    BASE_URL = "https://www.fantasypros.com/nfl/adp/overall.php"
elif ppr == 0.5:
    BASE_URL = "https://www.fantasypros.com/nfl/adp/half-point-ppr-overall.php"
elif ppr == 1:
    BASE_URL = "https://www.fantasypros.com/nfl/adp/ppr-overall.php"
else:
    print('URL could not be found')

In [6]:
print('Data will be scraped from: ' + BASE_URL)

Data will be scraped from: https://www.fantasypros.com/nfl/adp/ppr-overall.php


In [7]:
# function to scrape data from website
def make_adp_df():
    res = requests.get(BASE_URL)
    if res.ok:
        soup = BS(res.content, 'html.parser')
        table = soup.find('table', {'id': 'data'})
        df = pd.read_html(str(table))[0]
        print('Output after reading the html:\n\n', df.head(), '\n') # so you can see the output at this point
        df = df[['Player Team (Bye)', 'POS', 'AVG']]
        print('Output after filtering:\n\n', df.head(), '\n')
        df['BYE'] = df['Player Team (Bye)'].str.extract(r'(\d+)')
        df['PLAYER'] = df['Player Team (Bye)'].apply(lambda x: ' '.join(x.split()[:-2])) # removing the team and position
        df['POS'] = df['POS'].str.extract(r'(\D+)') # removing the position rank
        
        df = df[['PLAYER', 'POS', 'BYE', 'AVG']].sort_values(by='AVG')
        
        print('Final output: \n\n', df.head())
        
        return df
        
    else:
        print('oops, something didn\'t work right', res.status_code)
        
df = make_adp_df()

# remove positions that aren't of relvance to user and create positional data frames
if roster['QB'] > 0:
    qb_adp = df[df['POS']=='QB'] 
    qb_adp['POS RANK'] = qb_adp['AVG'].rank()
    qb_adp.index = qb_adp['POS RANK']
else:
    df = df[df['POS']!='QB']

if roster['RB'] > 0:
    rb_adp = df[df['POS']=='RB'] 
    rb_adp['POS RANK'] = rb_adp['AVG'].rank()
    rb_adp.index = rb_adp['POS RANK']
else:
    df = df[df['POS']!='RB']
    
if roster['WR'] > 0:
    wr_adp = df[df['POS']=='WR']
    wr_adp['POS RANK'] = wr_adp['AVG'].rank()
    wr_adp.index = wr_adp['POS RANK']
else:
    df = df[df['POS']!='WR']

if roster['TE'] > 0:
    te_adp = df[df['POS']=='TE'] 
    te_adp['POS RANK'] = te_adp['AVG'].rank()
    te_adp.index = te_adp['POS RANK']
else:
    df = df[df['POS']!='TE']

if roster['DST'] > 0:
    dst_adp = df[df['POS']=='DST']
    dst_adp['POS RANK'] = dst_adp['AVG'].rank()
    dst_adp.index = dst_adp['POS RANK']
else:
    df = df[df['POS']!='DST']

Output after reading the html:

    Rank             Player Team (Bye)  POS  ESPN  MFL  RTSports  Fantrax  AVG
0     1  Christian McCaffrey CAR (13)  RB1   1.0  NaN       1.0      1.0  1.0
1     2       Saquon Barkley NYG (11)  RB2   2.0  NaN       2.0      2.0  2.0
2     3      Ezekiel Elliott DAL (10)  RB3   3.0  NaN       3.0      4.0  3.3
3     4         Michael Thomas NO (6)  WR1   5.0  NaN       6.0      3.0  4.7
4     5           Alvin Kamara NO (6)  RB4   4.0  NaN       4.0      6.0  4.7 

Output after filtering:

               Player Team (Bye)  POS  AVG
0  Christian McCaffrey CAR (13)  RB1  1.0
1       Saquon Barkley NYG (11)  RB2  2.0
2      Ezekiel Elliott DAL (10)  RB3  3.3
3         Michael Thomas NO (6)  WR1  4.7
4           Alvin Kamara NO (6)  RB4  4.7 

Final output: 

                 PLAYER POS BYE  AVG
0  Christian McCaffrey  RB  13  1.0
1       Saquon Barkley  RB  11  2.0
2      Ezekiel Elliott  RB  10  3.3
3       Michael Thomas  WR   6  4.7
4         Alvin Kama

C:\Users\lewis\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lewis\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lewis\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

## Section 3: Finding the Replacement Players Based on User's Roster

In [8]:
if roster['DST'] > 0:
    replacement_players = {
    'QB':'', 
    'RB':'',
    'WR':'', 
    'TE':'', 
    'DST':''
    }
else:
    replacement_players = {
    'QB':'', 
    'RB':'',
    'WR':'', 
    'TE':''
    }
replacement_players

{'QB': '', 'RB': '', 'WR': '', 'TE': '', 'DST': ''}

In [28]:
rQB = np.ceil(0.9*numTeams*roster['QB'] + 1.1*numTeams*roster['QB/RB/WR/TE'])
rRB = np.ceil(0.9*numTeams*roster['RB'] + 0.2*numTeams*roster['QB/RB/WR/TE'] + 0.3*numTeams*roster['RB/WR/TE'] + 0.5*numTeams*roster['RB/WR'])
rWR = np.ceil(0.9*numTeams*roster['WR'] + 0.2*numTeams*roster['QB/RB/WR/TE'] + 0.27*numTeams*roster['RB/WR/TE'] + 0.5*numTeams*roster['RB/WR'])
rTE = np.ceil(0.45*numTeams*roster['TE'] + 0.01*numTeams*roster['QB/RB/WR/TE'] + 0.03*numTeams*roster['RB/WR/TE'])
if roster['DST'] > 0:
    rDST = np.ceil(numTeams*roster['DST'])

In [29]:
while rQB not in qb_adp.index:
    rQB = rQB + 0.5
if type(qb_adp.loc[rQB, 'PLAYER'])!=str:
    replacement_players['QB'] = qb_adp.loc[rQB, 'PLAYER'].iloc[0]
else:
    replacement_players['QB'] = qb_adp.loc[rQB, 'PLAYER']

    
while rRB not in rb_adp.index:
    rRB = rRB + 0.5
if type(rb_adp.loc[rRB, 'PLAYER'])!=str:
    replacement_players['RB'] = rb_adp.loc[rRB, 'PLAYER'].iloc[0]
else:
    replacement_players['RB'] = rb_adp.loc[rRB, 'PLAYER']

    
while rWR not in wr_adp.index:
    rWR = rWR + 0.5
if type(wr_adp.loc[rWR, 'PLAYER'])!=str:
    replacement_players['WR'] = wr_adp.loc[rWR, 'PLAYER'].iloc[0]
else:
    replacement_players['WR'] = wr_adp.loc[rWR, 'PLAYER']

while rTE not in te_adp.index:
    rTE = rTE + 0.5
if type(te_adp.loc[rTE, 'PLAYER'])!=str:
    replacement_players['TE'] = te_adp.loc[rTE, 'PLAYER'].iloc[0]
else:
    replacement_players['TE'] = te_adp.loc[rTE, 'PLAYER']

if roster['DST'] > 0:
    while rDST not in dst_adp.index:
        rDST = rDST + 0.5
    if type(dst_adp.loc[rDST, 'PLAYER'])!=str:
        replacement_players['DST'] = dst_adp.loc[rDST, 'PLAYER'].iloc[0]
    else:
        replacement_players['DST'] = dst_adp.loc[rDST, 'PLAYER']    

replacement_players

{'QB': 'Gardner Minshew II',
 'RB': 'Kareem Hunt',
 'WR': 'A.J. Green',
 'TE': 'Evan Engram',
 'DST': 'Los Angeles Rams'}

## Section 4: Scraping FantasyPros for Player Projections

In [30]:
# each position has a different associated URL. We'll create a string format here and loop through the possible positions
BASE_URL = 'https://www.fantasypros.com/nfl/projections/{position}.php?week=draft'

pos_list = [pos.lower() for pos in list(replacement_players.keys())]

def make_projection_df():
    
    # we are going to concatenate our individual position dfs into this larger final_df
    final_df = pd.DataFrame()
    
    # url has positions in lower case
    for position in pos_list:
        res = requests.get(BASE_URL.format(position=position)) # format our url with the position
        if res.ok:
            soup = BS(res.content, 'html.parser')
            table = soup.find('table', {'id':'data'})
            df = pd.read_html(str(table))[0]
                        
            if roster['DST']>0:
                if position == 'dst':
                    df.columns = df.columns
                else:
                    df.columns = df.columns.droplevel(level=0) # our data has a multi-level index and the first column level is useless
            else:
                df.columns = df.columns.droplevel(level=0)                
            
            if roster['DST']>0:
                if position == 'dst':
                    df['PLAYER'] = df['Player']
                else:
                    df['PLAYER'] = df['Player'].apply(lambda x: ' '.join(x.split()[:-1])) # fixing player name to not include team
            else:
                df['PLAYER'] = df['Player'].apply(lambda x: ' '.join(x.split()[:-1])) # fixing player name to not include team
          
            if 'REC' in df.columns:
                df['FPTS'] = df['FPTS'] + ppr*df['REC'] # add receptions if they are in there
            
            df['POS'] = position.upper() # add a position column
            
            df = df[['PLAYER', 'POS', 'FPTS']]
            print(df.head())
            final_df = pd.concat([final_df, df]) # iteratively add to final_df
            
        else:
            print('oops something went wrong', res.status_code)
            return
  
    final_df = final_df.sort_values(by='FPTS', ascending=False) # sort df in descending order on FPTS column
    
    return final_df
            
df = make_projection_df()
df.head()

            PLAYER POS   FPTS
0    Lamar Jackson  QB  355.2
1  Patrick Mahomes  QB  342.0
2     Dak Prescott  QB  314.1
3   Deshaun Watson  QB  312.1
4   Russell Wilson  QB  304.0
                PLAYER POS   FPTS
0  Christian McCaffrey  RB  366.8
1       Saquon Barkley  RB  306.5
2      Ezekiel Elliott  RB  299.2
3        Derrick Henry  RB  259.2
4          Dalvin Cook  RB  289.5
           PLAYER POS   FPTS
0  Michael Thomas  WR  326.4
1   Davante Adams  WR  287.5
2     Julio Jones  WR  277.9
3     Tyreek Hill  WR  257.3
4    Chris Godwin  WR  250.4
          PLAYER POS   FPTS
0   Travis Kelce  TE  254.2
1  George Kittle  TE  233.5
2   Mark Andrews  TE  189.9
3      Zach Ertz  TE  207.4
4  Darren Waller  TE  189.8
                 PLAYER  POS   FPTS
0   Pittsburgh Steelers  DST  120.6
1   San Francisco 49ers  DST  116.6
2      Los Angeles Rams  DST  113.6
3  New England Patriots  DST  111.3
4      Baltimore Ravens  DST  110.0


,PLAYER,POS,FPTS
0,Christian McCaffrey,RB,366.8
0,Lamar Jackson,QB,355.2
1,Patrick Mahomes,QB,342.0
0,Michael Thomas,WR,326.4
2,Dak Prescott,QB,314.1


## Section 5: Calculating Value Over Replacement (VOR)

In [31]:
# finding the projected fantasy points of the players in the replacement_players dictionary
if roster['DST'] > 0:
    replacement_values = {
        'QB': 0,
        'RB': 0,
        'WR': 0,
        'TE': 0,
        'DST':0
    }

else:
    replacement_values = {
        'QB': 0,
        'RB': 0,
        'WR': 0,
        'TE': 0
    }
    

for position, player in replacement_players.items():
    replacement_values[position] = df.loc[df['PLAYER'] == player].values[0,-1]
    
replacement_values

{'QB': 246.2, 'RB': 175.4, 'WR': 200.8, 'TE': 168.5, 'DST': 113.6}

In [32]:
df.POS.unique()

array(['RB', 'QB', 'WR', 'TE', 'DST'], dtype=object)

In [33]:
df['VOR'] = df.apply(
    lambda row: row['FPTS']-replacement_values.get(row['POS']), axis=1
)

In [34]:
# subtracting replacement value from projected fantasy points to calculate VOR
df['VOR'] = df.apply(
    lambda row: row['FPTS'] - replacement_values.get(row['POS']), axis=1
)
df['VOR/G'] = df['VOR']/16
df['VOR/G'] = df['VOR/G'].round(decimals=2)

df.head()

,PLAYER,POS,FPTS,VOR,VOR/G
0,Christian McCaffrey,RB,366.8,191.4,11.96
0,Lamar Jackson,QB,355.2,109.0,6.81
1,Patrick Mahomes,QB,342.0,95.8,5.99
0,Michael Thomas,WR,326.4,125.6,7.85
2,Dak Prescott,QB,314.1,67.9,4.24


In [35]:
# sort data frame by VOR
df = df.sort_values(by='VOR/G', ascending=False)
df['VALUERANK'] = df['VOR/G'].rank(ascending=False)
df['POS RANK'] = df.index+1
df.index = df.VALUERANK
df.drop('VALUERANK', axis=1, inplace = True)
df = df[:playersdrafted+50]
df.head(24)

,PLAYER,POS,FPTS,VOR,VOR/G,POS RANK
VALUERANK,,,,,,
1.0,Christian McCaffrey,RB,366.8,191.4,11.96,1
2.0,Saquon Barkley,RB,306.5,131.1,8.19,2
3.0,Michael Thomas,WR,326.4,125.6,7.85,1
4.0,Ezekiel Elliott,RB,299.2,123.8,7.74,3
5.0,Dalvin Cook,RB,289.5,114.1,7.13,5
6.0,Alvin Kamara,RB,287.6,112.2,7.01,6
7.0,Lamar Jackson,QB,355.2,109.0,6.81,1
8.0,Patrick Mahomes,QB,342.0,95.8,5.99,2
9.0,Davante Adams,WR,287.5,86.7,5.42,2


In [17]:
download = input('Download cheatsheet? (y/n) ').lower()
if download == 'y':
    df.to_excel(leagueName +' Cheatsheet.xlsx')
else:
    pass

Download cheatsheet? (y/n) y
